In [1]:
!git clone https://github.com/workpiles/CUCUMBER-9.git

fatal: destination path 'CUCUMBER-9' already exists and is not an empty directory.


In [2]:
# ファイルを解凍する。
# prototype_1
!tar -zxvf ./CUCUMBER-9/prototype_1/cucumber-9-python.tar.gz

batches.meta
data_batch_5
data_batch_4
data_batch_3
data_batch_2
data_batch_1
test_batch


In [0]:
!mkdir weights

In [4]:
import numpy as np
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, BatchNormalization, LeakyReLU 
from keras.layers import Reshape, Conv2DTranspose, Activation, MaxPooling2D
from keras.layers import Flatten, Dropout, Lambda
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [0]:
def load_image_and_label(pickled_files):
    # 各ファイルには495画像が含まれている
    IMAGE_COUNT_PER_FILE = 495
    # 画像の形状 : 32x32x3
    ROW = 32
    COL = 32
    DIM = 3
    whole_images = np.empty((IMAGE_COUNT_PER_FILE*len(pickled_files), ROW, COL, DIM))
    whole_labels = np.empty(IMAGE_COUNT_PER_FILE*len(pickled_files))
    for i, pickled_file in enumerate(pickled_files):
        dict = _unpickle(pickled_file)
        images = dict['data'].reshape(IMAGE_COUNT_PER_FILE, DIM, ROW, COL).transpose(0, 2, 3, 1)
        whole_images[i*IMAGE_COUNT_PER_FILE:(i + 1)*IMAGE_COUNT_PER_FILE, :, :, :] = images
        labels = dict['labels']
        whole_labels[i*IMAGE_COUNT_PER_FILE:(i + 1)*IMAGE_COUNT_PER_FILE] = labels
    return (whole_images, whole_labels)

In [0]:
def _unpickle(pickled_file):
    import pickle

    with open(pickled_file, 'rb') as file:
        dict = pickle.load(file, encoding='latin1')
    return dict

In [0]:
def load_cucumber():
    (X_train, y_train) = load_image_and_label(['data_batch_1',
                                     'data_batch_2',
                                     'data_batch_3',
                                     'data_batch_4',
                                     'data_batch_5',])
    (X_test, y_test) = load_image_and_label(['test_batch'])
    
    return X_train, X_test, y_train, y_test

In [0]:
def normalize(X):
    return (X-127.5)/127.5

In [0]:
### データの準備
X_train, X_test, y_train, y_test = load_cucumber()
X_train = normalize(X_train)
X_test = normalize(X_test)
input_shape = X_train[0].shape
X_test_original = X_test.copy()

y_train = np.eye(9)[y_train.astype('int32').flatten()]
y_test = np.eye(9)[y_test.astype('int32').flatten()]


In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2475, 32, 32, 3)
(2475, 9)
(495, 32, 32, 3)
(495, 9)


In [0]:
model = Sequential()

In [0]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal', input_shape=(32, 32, 3)))
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal'))
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',
                 kernel_initializer='he_normal'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',
                 kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())

model.add(Flatten())  
model.add(Dense(1024, activation='relu',
                kernel_initializer='he_normal')) 
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))  
model.add(Dense(9, activation='softmax'))

In [128]:
from keras.applications.vgg16 import VGG16
model = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)

553467904/553467096 [==============================] - 6s 0us/step


In [0]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='Adam',
    metrics=['accuracy']
)

In [130]:
early_stopping = EarlyStopping(patience=1, verbose=1)
model.fit(x=X_train, y=y_train, batch_size=256, epochs=10, verbose=1,
          validation_data=(X_test, y_test), callbacks=[early_stopping])

ValueError: ignored

In [127]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.537748224807508
Test accuracy: 0.42828282870427525
